<a href="https://colab.research.google.com/github/osgeokr/kari-sdm/blob/main/006_scaled_minmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 43.1 MB/s eta 0:00:00


In [3]:
import os
import rasterio
from rasterio.enums import Resampling
from rasterio.shutil import copy
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from google.colab import drive

In [4]:
# Google Drive를 마운트
drive.mount('/content/drive')

# 경로 설정
folder_path = "/content/drive/My Drive/kari-sdm"
files = [
    "DEM.tif",
    "SLP.tif",
    "ASP.tif",
    "NDBI.tif",
    "NDVI.tif",
    "NDWI.tif",
    "CHT.tif",
    "TCC.tif",
]

Mounted at /content/drive


In [5]:
# 파일 처리 및 저장
for file_name in files:
    file_path = os.path.join(folder_path, file_name)

    with rasterio.open(file_path) as src:
        # 첫 번째 밴드를 읽어 스케일링 수행
        data = src.read(1)
        scaler = MinMaxScaler()
        data_scaled = scaler.fit_transform(data.reshape(-1, 1)).reshape(data.shape)

        # 스케일링된 파일 이름 설정 (e.g., DEM_scaled.tif)
        base_name = os.path.splitext(file_name)[0]
        scaled_file_name = f"{base_name}_scaled_temp.tif"
        scaled_file_path = os.path.join(folder_path, scaled_file_name)

        # COG로 저장하기 위한 메타데이터 설정
        profile = src.profile
        profile.update(
            dtype=rasterio.float32,
            count=1,
            compress='lzw'
        )

        # 스케일링된 데이터를 임시 파일에 저장
        with rasterio.open(scaled_file_path, 'w', **profile) as dst:
            dst.write(data_scaled.astype(rasterio.float32), 1)

        # COG로 변환하여 원래 원하는 파일 이름으로 저장
        final_scaled_file_path = os.path.join(folder_path, f"{base_name}_scaled.tif")
        os.rename(scaled_file_path, final_scaled_file_path)

        print(f"Processed and saved {final_scaled_file_path} as COG")

Processed and saved /content/drive/My Drive/kari-sdm/DEM_scaled.tif as COG
Processed and saved /content/drive/My Drive/kari-sdm/SLP_scaled.tif as COG
Processed and saved /content/drive/My Drive/kari-sdm/ASP_scaled.tif as COG
Processed and saved /content/drive/My Drive/kari-sdm/NDBI_scaled.tif as COG
Processed and saved /content/drive/My Drive/kari-sdm/NDVI_scaled.tif as COG
Processed and saved /content/drive/My Drive/kari-sdm/NDWI_scaled.tif as COG
Processed and saved /content/drive/My Drive/kari-sdm/CHT_scaled.tif as COG
Processed and saved /content/drive/My Drive/kari-sdm/TCC_scaled.tif as COG
